In [1]:
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Conv2D, LeakyReLU, UpSampling2D, InputLayer, Concatenate, Input, merge, concatenate, Lambda, Reshape
from keras.layers.normalization import BatchNormalization
from keras.utils import plot_model

import tensorflow as tf

Using TensorFlow backend.


In [2]:
def parse_config(cfg_path, verbose=False):
    file = open(cfg_path, 'r')
    lines = file.readlines()
        
    # get rid of comments and blank lines and white space
    lines = [x for x in lines if len(x) > 1]
    lines = [x for x in lines if x[0] != '#']
    lines = [x.rstrip().lstrip() for x in lines]

    block = {}
    blocks = []
    
    for line in lines:
        if verbose: print('valuating line: %s' % line)
        
        if line[0] == '[': # new block start
            if len(block) != 0: # if the block inst empty (has data) then reset it
                blocks.append(block)
                block = {}
                
            block['type'] = line[1:-1]
        else:
            key, value = line.split('=')
            key, value = key.rstrip(), value.lstrip()
            block[key] = value

    return blocks

In [3]:
## The old network creator

In [4]:
main_modle = Sequential()

all_modules = [Input((416, 416, 3))] # add InputLayer for sequential
module = all_modules[-1]

shortcut = None

for index, block in enumerate(blocks[1:]): # block 0 specifies network info
    print('%s, index: %i/%i' % (block['type'], index, len(blocks)))
    if index > 20: continue
    
    ### Conv(ish) layers
    if block['type'] == 'convolutional':
        # layer config
        activation = block['activation']
        
        bais = False
        try:
            batch_normalize = int(block['batch_normalize'])
        except:
            batch_normalize = 0
            bais = True
            
        filters = int(block['filters'])
        padding = int(block['pad'])
        kerel_size = int(block['size'])
        strides = int(block['stride'])
        
# not working in keras
#         if padding:
#             pad = (kernel_size - 1) // 2
#         else:
#             pad = 0
        
        # create the conv layer
        module = Conv2D(filters, (kerel_size, kerel_size), strides=(strides, strides), use_bias=bais)(module)
        
        if batch_normalize:
            module = BatchNormalization()(module)
        
        if activation == 'leaky':
            module = LeakyReLU(0.1)(module)
            
        if index < 1:
            module = MaxPooling2D(pool_size=(2, 2))(module)
            
    elif block['type'] == 'upsample':
        stride = int(block['stride'])
        module = UpSampling2D(size=(stride, stride), interpolation='bilinear')(module)

    ### Routs and shortcuts    
    elif block['type'] == 'route': # we need to connect these in the right way
        layers = block['layers'].split(',')
        start = int(layers[0])
        
        try:
           end = int(blcok['layers'][1]) 
        except: # if there is no end
            end = 0
        
        if start > 0:
            start = start - index
        if end > 0:
            end = end - index
        
#         from IPython.core.debugger import Tracer; Tracer()()
        
        print('start: %d, end: %d, all_modules: [%s]' % (start, end, '\n'.join(map(str, all_modules[index + start - 1: index + end]))))
    
        l1 = all_modules[index + start]
        l2 = all_modules[index + end]
        
        if l1.shape[1] == l2.shape[1]: # hack to make sure tensors fit
            module = concatenate([l1, l2])
    
    elif block['type'] == 'shortcut':
        f = int(block['from'])
        module = all_modules[index + f] # this is not great, but will be okay for now
        
#         module = InputLayer(all_modules[index + f].shape)([module, all_modules[index + f]])
        
#         from IPython.core.debugger import Tracer; Tracer()()
#         shortcut = all_modules[index + f]
#         module = concatenate([shortcut, module])

    elif block['type'] == 'yolo': # this needs to also be connected to the network
        mask = block['mask'].split(',')
        mask = [int(x) for x in mask] # convert to ints
        
        anchors = block['anchors'].split(',')
        anchors = [int(a) for a in anchors]
        anchors = [(anchors[i], anchors[i + 1]) for i in range(0, len(anchors), 2)] # break anchors into pairs
        anchors = [anchors[i] for i in mask]
    
    all_modules += [module]

all_modules

NameError: name 'blocks' is not defined